## Project description.

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

I've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

### Table of contents:

**[Connecting to the Database](#Connecting-to-the-Database:)**  <br>

**[Data diagram](#Data-diagram:)**

**[Description of the data](#Description-of-the-data:)**

 - [books](#books:)
 - [authors](#authors:)
 - [publishers](#publishers:)
 - [ratings](#ratings:)
 - [reviews](#reviews:) 

**[Tasks:](#Tasks:)**

 - [Number of books released after January 1, 2000](#Number-of-books-released-after-January-1,-2000:)
 - [Number of user reviews and the average rating for each book](#Number-of-user-reviews-and-the-average-rating-for-each-book:)
 - [Publisher that has released the greatest number of books with more than 50 pages](#Publisher-that-has-released-the-greatest-number-of-books-with-more-than-50-pages:)
 - [Author with the highest average book rating](#Author-with-the-highest-average-book-rating:)
 - [Average number of text reviews among users who rated more than 50 books](#Average-number-of-text-reviews-among-users-who-rated-more-than-50-books:)

In [1]:
!pip install psycopg2 -q
import pandas as pd
from sqlalchemy import create_engine

## Connecting to the Database:

In [2]:
db_config = {'user': 'practicum_student', # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # connection port
             'db': 'data-analyst-final-project-db'} # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Data diagram:

![njkklkn](https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-02T142019.920_1593688954.png)

## Description of the data:

### books:

- `book_id`
- `author_id`
- `title`
- `num_pages` — number of pages
- `publication_date`
- `publisher_id`

In [3]:
query = ''' SELECT *
            FROM books
            LIMIT 3
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


### authors:
- `author_id`
- `author`

In [4]:
query = ''' SELECT *
            FROM authors
            LIMIT 3
        '''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


### publishers:

- `publisher_id`
- `publisher`

In [5]:
query = ''' SELECT *
            FROM publishers
            LIMIT 3
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


### ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

In [6]:
query = ''' SELECT *
            FROM ratings
            LIMIT 3
        '''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


### reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book
- `text` — the text of the review

In [7]:
query = ''' SELECT *
            FROM reviews
            LIMIT 3
        '''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


## Tasks:

### Number of books released after January 1, 2000:

In [8]:
query = ''' SELECT COUNT(book_id) AS number_of_books
            FROM books
            WHERE publication_date > '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine)

,number_of_books
0,819


**819** books were released after January 1, 2000.

### Number of user reviews and the average rating for each book:

In [9]:
query = ''' SELECT books.title AS title,
                   COUNT(DISTINCT reviews.review_id) AS user_reviews_count,
                   AVG(ratings.rating) AS average_rating
            FROM   books
                   LEFT JOIN reviews ON books.book_id = reviews.book_id
                   LEFT JOIN ratings ON books.book_id = ratings.book_id
            GROUP BY
                   books.book_id
            ORDER BY 
                   books.title
        '''

pd.io.sql.read_sql(query, con = engine)

,title,user_reviews_count,average_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


### Publisher that has released the greatest number of books with more than 50 pages:
This will help us to exclude brochures and similar publications from our analysis:

In [10]:
query = ''' SELECT publishers.publisher AS publisher,
                   COUNT(books.book_id) AS books_count
            FROM   publishers
                   LEFT JOIN books ON books.publisher_id = publishers.publisher_id 
            WHERE  books.num_pages > 50        
            GROUP BY
                   publishers.publisher_id
            ORDER BY
                   books_count DESC 
            LIMIT 1       
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher,books_count
0,Penguin Books,42


**Penguin Books** publisher has released the greatest number of books among the others, which is **42** books.

### Author with the highest average book rating:
We'll look only at books with at least 50 ratings

In [11]:
query = ''' SELECT authors.author AS author,
                   average_book_rating
            FROM   authors
                   LEFT JOIN books ON authors.author_id = books.author_id
                   RIGHT JOIN (SELECT books.book_id AS book_id,
                                      ROUND(AVG(ratings.rating), 1) AS average_book_rating
                              FROM    books
                                      LEFT JOIN ratings ON books.book_id = ratings.book_id
                              GROUP BY
                                      books.book_id
                              HAVING  COUNT(ratings.rating_id) >= 50) AS avg_book_r
                                   ON books.book_id = avg_book_r.book_id
            ORDER BY
                   average_book_rating DESC  
            LIMIT 1                         
        '''

pd.io.sql.read_sql(query, con = engine)

,author,average_book_rating
0,J.K. Rowling/Mary GrandPré,4.4


**J.K. Rowling / Mary GrandPré** is the author with the highest average book rating (for books that have at least 50 ratings). <br>
Average book rating for their highest rated book is **4.4**.

### Average number of text reviews among users who rated more than 50 books:

In [12]:
query = ''' SELECT ROUND(AVG(text_reviews_count)) AS average_text_reviews_count
            FROM (SELECT ratings.username AS username,
                         COUNT(ratings.book_id)
                  FROM   ratings
                         LEFT JOIN books ON ratings.book_id = books.book_id 
                  GROUP BY 
                         ratings.username
                  HAVING COUNT(ratings.book_id) > 50) 
                  AS A
            LEFT JOIN (SELECT reviews.username AS username,
                              COUNT(reviews.text) AS text_reviews_count
                       FROM   reviews
                       GROUP BY
                              reviews.username) 
                  AS B
            ON A.username = B.username         
        '''

pd.io.sql.read_sql(query, con = engine)

,average_text_reviews_count
0,24.0


Average number of text reviews among users who rated more than 50 books is **24**.